In [11]:
import pandas as pd
import numpy as np
import math
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler, MinMaxScaler


class ModelFirst:
    def model1(self, selected_genres):
        df = pd.read_csv('/content/drive/MyDrive/data/user_pref.csv')
        new_movie_data = pd.read_csv('/content/drive/MyDrive/data/movieId&title.csv')
        new_movie_data = new_movie_data[['id','genres']]

        user_pref = df.to_numpy()
        transformer = StandardScaler()
        transformer.fit(user_pref)
        standard_user_pref = transformer.transform(user_pref)
        # print(standard_user_pref.shape)
        df = pd.DataFrame(user_pref, columns=df.columns)
        # print(df.head(3))
        
        first_genre_index = df[selected_genres[0]].sort_values(ascending=False).head(1400).index
        second_genre_index = df[selected_genres[1]].sort_values(ascending=False).head(1400).index
        
        common_index = []
        for idx in first_genre_index:
            if idx in second_genre_index:
                common_index.append(idx)

        common_df = pd.DataFrame(index=range(0,len(common_index)), columns=df.columns)
        i = 0
        for idx in common_index:
            common_df.iloc[i] = df.iloc[idx]
            i = i + 1

        common_np = common_df.to_numpy()
        # 장르 유사도 계산
        # vector = TfidfVectorizer()
        # trans_data = np.array(new_movie_data['genres'].tolist())
        # movie_genre_tfidf = vector.fit_transform(trans_data).toarray()

        # SVD = TruncatedSVD(n_components=8)
        # matrix = SVD.fit_transform(movie_genre_tfidf.T)
        # matrix.shape
        corr = np.load('/content/drive/MyDrive/data/corr_save.npy')

        genre_title = df.columns
        genre_list = list(genre_title)
        first_corr_idx = genre_list.index(selected_genres[0])
        second_corr_idx = genre_list.index(selected_genres[1])
        common_corr = corr[first_corr_idx][second_corr_idx]

        result_np = common_np * common_corr
        result_df = pd.DataFrame(result_np, columns=genre_title)
        result = result_df.max().idxmax()
        
        while(True): 
            if result in selected_genres:
                result_df.drop(result, axis= 1, inplace=True)
                result = result_df.max().idxmax()
            else:
                break

        temp = selected_genres
        result = list(result.split())
        result = temp+result
        return result


In [12]:
# c = ModelFirst()
# selected_genre = ['Animation', 'Drama']
# c.model1(selected_genre)

     Action  Adventure  Animation  ...  Thriller       War  Western
0  1.129182   0.890254   0.632951  ...       0.0  0.700709      0.0
1  1.286950   1.665838   2.116452  ...       0.0  0.000000      0.0
2  2.382003   0.496014   0.000000  ...       0.0  0.000000      0.0

[3 rows x 22 columns]


['Animation', 'Drama', 'Comedy']